<a href="https://colab.research.google.com/github/M123shashank/Predicting-Coal-Stock-Levels/blob/main/Predicting_Coal_Stock_Levels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.base import clone
from xgboost import XGBRegressor

from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score

# 1. Load daily data
df = pd.read_csv("/content/drive/MyDrive/Datasets/daily-coal-stocks.csv", dtype={'utility': 'str'},
                 parse_dates=["date"])

# 2. Data Cleaning
# Fill 'unknown' in place of None in 'utility'
df['utility'] = df['utility'].fillna('unknown')

# Remove rows with missing capacity_in_mw
missing_capacity_rows = df[df['capacity_in_mw'].isna()]
df.dropna(subset=['capacity_in_mw'], inplace=True)

# Fill missing 'req_for_day_in_th_tonnes' using normative stock / stock days if required features not missing
df = df.dropna(subset=['req_for_day_in_th_tonnes', 'normative_stock_req_in_th_tonnes'], how='all')
mask_4 = df['req_for_day_in_th_tonnes'].isna() & df['normative_stock_req_in_th_tonnes'].notna() & df['normative_stock_req_days'].notna()
df.loc[mask_4, 'req_for_day_in_th_tonnes'] = (df.loc[mask_4, 'normative_stock_req_in_th_tonnes'] / df.loc[mask_4, 'normative_stock_req_days']
)

# Drop rows where 'normative_stock_req_days' is missing and neither normative stock nor percentage is available
mask_5 = (df['normative_stock_req_days'].isna() & df['normative_stock_req_in_th_tonnes'].isna() &
          df['percentage_of_actual_stock_vs_normative_stock'].isna())
df = df[~mask_5]

# Fill normative stock where daily req and days are known
mask_6 = df['normative_stock_req_in_th_tonnes'].isna() & df['req_for_day_in_th_tonnes'].notna() & df['normative_stock_req_days'].notna()
df.loc[mask_6, 'normative_stock_req_in_th_tonnes'] = (
    df.loc[mask_6, 'req_for_day_in_th_tonnes'] * df.loc[mask_6, 'normative_stock_req_days'])

# Calculate actual stock days as well as percentage_of_actual_stock_vs_normative_stock
mask_8 = df['actual_stock_total_in_th_tonnes'].notna() & df['req_for_day_in_th_tonnes'].notna()
df.loc[mask_8, 'actual_stock_days'] = (
    df.loc[mask_8, 'actual_stock_total_in_th_tonnes'] / df.loc[mask_8, 'req_for_day_in_th_tonnes'])

mask_9 = df['actual_stock_total_in_th_tonnes'].notna() & df['normative_stock_req_in_th_tonnes'].notna()
df.loc[mask_9, 'percentage_of_actual_stock_vs_normative_stock'] = (
    100 * df.loc[mask_9, 'actual_stock_total_in_th_tonnes'] / df.loc[mask_9, 'normative_stock_req_in_th_tonnes'])

# Drop the last two columns, which are of no use and do some basic preprocessing
df = df.drop(columns=['critical_or_sup_critical','reasons_for_critical_remarks'], errors='ignore')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Clean transport modes
df['mode_of_transport'] = df['mode_of_transport'].str.strip().replace({
    'Raiil': 'Rail', 'Rail-Sea': 'Sea', 'Rail-Sea-1500': 'Sea',
    'Rail-Sea ': 'Sea', 'Rail-Sea-Rail': 'Sea', 'Rail-1500': 'Rail'})

# 3. Aggregate to monthly level
monthly = df.groupby(['year','month']).agg(
    avg_stock=('actual_stock_total_in_th_tonnes','mean'),
    std_stock=('actual_stock_total_in_th_tonnes','std'),
    total_stock=('actual_stock_total_in_th_tonnes','sum'),
    count_plants=('power_station_name','nunique')
).reset_index()

# Do Sector-level & Transport-mode totals and merge with monthly
sector_totals = df.groupby(['year','month','sector'])['actual_stock_total_in_th_tonnes'] \
                  .sum().unstack(fill_value=0).reset_index()
mode_totals = df.groupby(['year','month','mode_of_transport'])['actual_stock_total_in_th_tonnes'] \
                .sum().unstack(fill_value=0).reset_index()

monthly = monthly.merge(sector_totals, on=['year','month'], how='left') \
                 .merge(mode_totals, on=['year','month'], how='left').fillna(0)

# Create fractional features
monthly['frac_central']    = monthly.get('Central Sector', 0) / monthly['total_stock']
monthly['frac_state']      = monthly.get('State Sector', 0) / monthly['total_stock']
monthly['frac_pvt']        = monthly.get('Pvt Sector', 0) / monthly['total_stock']
monthly['frac_joint']      = monthly.get('Joint Venture', 0) / monthly['total_stock']
monthly['frac_rail']       = monthly.get('Rail', 0) / monthly['total_stock']
monthly['frac_road']       = monthly.get('Road', 0) / monthly['total_stock']
monthly['frac_sea']        = monthly.get('Sea', 0) / monthly['total_stock']
monthly['frac_pithead']    = monthly.get('Pithead', 0) / monthly['total_stock']
monthly['frac_intermodal'] = monthly.get('Inter Modal', 0) / monthly['total_stock']

# Month cyclic encoding
monthly['month_sin'] = np.sin(2 * np.pi * monthly['month'] / 12)
monthly['month_cos'] = np.cos(2 * np.pi * monthly['month'] / 12)

# 4. Add target: next year's stock and binary label
def get_next_year_stock(row):
    mask = (monthly['year'] == row['year'] + 1) & (monthly['month'] == row['month'])
    next_val = monthly[mask]['avg_stock']
    return next_val.values[0] if not next_val.empty else np.nan

monthly.loc[:,'next_year_stock'] = monthly.apply(get_next_year_stock, axis=1)
monthly = monthly.dropna(subset=['next_year_stock'])
calculated_label = (monthly['next_year_stock'] >= 1.1 * monthly['avg_stock']).astype(int)
monthly.loc[:, 'label'] = calculated_label

# 5. Feature matrix
features = ['avg_stock', 'std_stock', 'count_plants',
            'frac_central', 'frac_state', 'frac_pvt', 'frac_joint',
            'frac_rail', 'frac_road', 'frac_sea', 'frac_pithead', 'frac_intermodal',
            'month_sin', 'month_cos']
X = monthly[features]
y_cls = monthly['label']
y_reg = monthly['next_year_stock']

# 6. Train/test split
X_train, X_test, y_train_cls, y_test_cls, y_train_reg, y_test_reg = train_test_split(
    X, y_cls, y_reg, test_size=0.2, random_state=42, stratify=y_cls)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

# 7. Base regressors
base_models = [
    ('Linear', LinearRegression()),
    ('XGB',    XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')),
    ('SVM', SVR(kernel = 'poly', C=1000.0)),
    ('NN',     MLPRegressor(hidden_layer_sizes=(500,), activation='logistic', solver='lbfgs', max_iter=8000, random_state=0))
]

# Cross-validated out-of-fold predictions
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros((X_train.shape[0], len(base_models)))
test_preds = np.zeros((X_test.shape[0], len(base_models)))

for i, (name, model) in enumerate(base_models):
    oof = np.zeros(X_train.shape[0])
    for train_idx, valid_idx in skf.split(X_train_scaled, y_train_cls):
        model_clone = clone(model)
        model_clone.fit(X_train_scaled[train_idx], y_train_reg.iloc[train_idx])
        oof[valid_idx] = model_clone.predict(X_train_scaled[valid_idx])
    oof_preds[:, i] = oof
    # Final test predictions
    model.fit(X_train_scaled, y_train_reg)
    test_preds[:, i] = model.predict(X_test_scaled)

# Train a meta-model on out-of-fold predictions for regression
reg_meta_model = LinearRegression()
reg_meta_model.fit(oof_preds, y_train_reg)

# Predict on test data using stacked model
final_reg_preds = reg_meta_model.predict(test_preds)

# Evaluate
print(f"R² Score:             {r2_score(y_test_reg, final_reg_preds):.4f}")
print(f"Mean Absolute Error:  {mean_absolute_error(y_test_reg, final_reg_preds):.4f}")

# 9. Classification with stacked predictions
meta_model = LogisticRegression( class_weight='balanced', solver = 'liblinear', max_iter=1000, random_state=42)
meta_model.fit(oof_preds, y_train_cls)
final_cls_preds = meta_model.predict(test_preds)

# 10. Classification evaluation
print("\nClassification Performance:")
print("Accuracy :", accuracy_score(y_test_cls, final_cls_preds))
print("Precision:", precision_score(y_test_cls, final_cls_preds))
print("Recall   :", recall_score(y_test_cls, final_cls_preds))

R² Score:             0.7319
Mean Absolute Error:  16.4640

Classification Performance:
Accuracy : 0.8
Precision: 0.8333333333333334
Recall   : 0.8333333333333334
